# Modelos de Clasificación – Nivel de gasto del cliente

## 1) Objetivo
Entrenar y evaluar modelos de Machine Learning que permitan clasificar
a los clientes según su nivel de gasto (Bajo, Medio o Alto), apoyando
decisiones de segmentación y estrategias comerciales.

## 2) Importación de librerías

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix
)

## 3) Carga del dataset procesado

In [2]:
df = pd.read_csv('../data/processed/clientes_clasificacion.csv')
df.head()

,Edad,Genero,Categoria_Producto,Metodo_Pago,Shopping_Mall,Nivel_Gasto
0,28,Female,Clothing,Credit Card,Kanyon,Alto
1,21,Male,Shoes,Debit Card,Forum Istanbul,Alto
2,20,Male,Clothing,Cash,Metrocity,Medio
3,66,Female,Shoes,Credit Card,Metropol AVM,Alto
4,53,Female,Books,Cash,Kanyon,Bajo


## 4) Definición de variables predictoras y variable objetivo

In [3]:
X = df.drop('Nivel_Gasto', axis=1)
y = df['Nivel_Gasto']

## 5) Identificación de variables numéricas y categóricas

In [4]:
num_features = ['Edad']
cat_features = [
    'Genero',
    'Categoria_Producto',
    'Metodo_Pago',
    'Shopping_Mall'
]

## 6) Preprocesamiento de datos

- Escalado de variables numéricas
- One-Hot Encoding de variables categóricas

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

## 7) División del dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 8.1) Modelo 1 – Logistic Regression

In [7]:
log_reg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000
    ))
])

log_reg_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

## 8.1.1) Evaluación del modelo – Logistic Regression

In [8]:
y_pred_log = log_reg_pipeline.predict(X_test)

accuracy_log = accuracy_score(y_test, y_pred_log)
print(f"Accuracy: {accuracy_log:.3f}")

Accuracy: 0.723


In [9]:
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

        Alto       0.66      1.00      0.80      6560
        Bajo       0.96      0.71      0.81      6729
       Medio       0.61      0.46      0.53      6603

    accuracy                           0.72     19892
   macro avg       0.74      0.72      0.71     19892
weighted avg       0.74      0.72      0.71     19892



In [10]:
confusion_matrix(y_test, y_pred_log)

array([[6560,    0,    0],
       [   0, 4751, 1978],
       [3333,  206, 3064]])

## 8.2) Modelo 2 – Random Forest Classifier

In [11]:
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
    ))
])

rf_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

## 8.2.1) Evaluación del modelo – Random Forest

In [12]:
y_pred_rf = rf_pipeline.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {accuracy_rf:.3f}")


Accuracy: 0.674


In [13]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

        Alto       0.67      0.81      0.73      6560
        Bajo       0.82      0.78      0.80      6729
       Medio       0.51      0.43      0.47      6603

    accuracy                           0.67     19892
   macro avg       0.67      0.67      0.67     19892
weighted avg       0.67      0.67      0.67     19892



In [14]:
confusion_matrix(y_test, y_pred_rf)

array([[5327,    0, 1233],
       [   0, 5253, 1476],
       [2616, 1164, 2823]])

## 9) Comparación de modelos

In [15]:
pd.DataFrame({
    'Modelo': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [accuracy_log, accuracy_rf]
})

,Modelo,Accuracy
0,Logistic Regression,0.722652
1,Random Forest,0.673788


- Random Forest suele capturar mejor relaciones no lineales.
- Logistic Regression ofrece mayor interpretabilidad.
- Se entrenaron modelos de clasificación para segmentar clientes
por nivel de gasto, evitando fuga de información y garantizando
un modelo útil y defendible para el negocio.
- En este caso, el accuracy del modelo LR es mayor por lo que es el potencial modelo elegido para predecir el nivel de gasto del cliente.

## 10) Predicción de gasto – Ejemplos

In [16]:
ejemplos = pd.DataFrame({
    'Edad': [22, 40, 65],
    'Genero': ['Female', 'Male', 'Female'],
    'Categoria_Producto': ['Clothing', 'Technology', 'Shoes'],
    'Metodo_Pago': ['Cash', 'Credit Card', 'Debit Card'],
    'Shopping_Mall': ['Forum Istanbul', 'Mall of Istanbul', 'Istinye Park']
})

pred_log = log_reg_pipeline.predict(ejemplos)
pred_rf = rf_pipeline.predict(ejemplos)

resultados = ejemplos.copy()
resultados['Pred_LogReg'] = pred_log
resultados['Pred_RF'] = pred_rf

resultados


,Edad,Genero,Categoria_Producto,Metodo_Pago,Shopping_Mall,Pred_LogReg,Pred_RF
0,22,Female,Clothing,Cash,Forum Istanbul,Alto,Alto
1,40,Male,Technology,Credit Card,Mall of Istanbul,Alto,Medio
2,65,Female,Shoes,Debit Card,Istinye Park,Alto,Alto


Los modelos permiten estimar el nivel de gasto esperado de un cliente
antes o durante la compra, facilitando la planificación de ingresos
y la personalización de estrategias comerciales.

Respecto al segundo ejemplo, se observa que ambos modelos clasifican el nivel de gasto de manera distinta.
Por ende, en base al accuracy, la predicción del modelo de LR sería más confiable que la del RF. 

## 11) Importancia de variables

La importancia de variables permite entender qué características influyen más
en la clasificación del nivel de gasto del cliente, aportando interpretabilidad
al modelo.

In [83]:
feature_names = (
    num_features +
    list(rf_pipeline.named_steps['preprocessor']
         .named_transformers_['cat']
         .get_feature_names_out(cat_features))
)

importances = rf_pipeline.named_steps['classifier'].feature_importances_

pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)


,Feature,Importance
0,Edad,0.241660
6,Categoria_Producto_Food & Beverage,0.196702
4,Categoria_Producto_Clothing,0.174226
7,Categoria_Producto_Shoes,0.098500
5,Categoria_Producto_Cosmetics,0.066360
8,Categoria_Producto_Souvenir,0.061569
9,Categoria_Producto_Technology,0.054081
10,Categoria_Producto_Toys,0.048141
3,Categoria_Producto_Books,0.033498
11,Metodo_Pago_Cash,0.002203


- La edad del cliente es un factor relevante para diferenciar niveles de gasto.
- Ciertas categorías de producto tienen mayor influencia en el gasto esperado.
- El método de pago y el centro comercial aportan información sobre el perfil
  económico del cliente.

## 12) Feature Engineering extra

Vamos a aplicar ingenería extra a las características para intentar mejorar el accuracy de los modelos.

Objetivo -> Crear variables más informativas que permitan al modelo:

- Capturar patrones no lineales

- Reducir ruido

- Generalizar mejor

### 12.1) Agrupación de edades

Por qué hacerlo:

- La edad exacta es ruidosa

- El comportamiento de compra suele cambiar por etapas de vida

- Mejora modelos no lineales y lineales

In [ ]:
# Creación de grupos etarios
df_fe = df.copy()

df_fe['Grupo_Edad'] = pd.cut(
    df_fe['Edad'],
    bins=[0, 25, 35, 45, 55, 100],
    labels=['18-25', '26-35', '36-45', '46-55', '55+']
)

In [19]:
df_fe['Grupo_Edad'].value_counts()

Grupo_Edad
55+      26587
36-45    19436
26-35    19059
46-55    19016
18-25    15359
Name: count, dtype: int64

La segmentación por grupos etarios permite identificar comportamientos
de gasto asociados a etapas de vida, facilitando estrategias de marketing
más personalizadas.

### 12.2) Reducción de cardinalidad en categorías

Problema:

- Muchas categorías → muchas columnas OneHot

- Riesgo de overfitting

In [23]:
# Agrupar categorías poco frecuentes
top_categories = df_fe['Categoria_Producto'].value_counts().nlargest(5).index

df_fe['Categoria_Producto'] = df_fe['Categoria_Producto'].apply(
    lambda x: x if x in top_categories else 'Otros'
)

In [43]:
df_fe['Categoria_Producto'].value_counts()

Categoria_Producto
Clothing           34487
Cosmetics          15097
Otros              14976
Food & Beverage    14776
Toys               10087
Shoes              10034
Name: count, dtype: int64

Agrupar categorías de baja frecuencia reduce el ruido y permite
enfocar el análisis en los productos más relevantes para el negocio.


### 12.3) Actualizar variables predictoras y el preprocesamiento

In [44]:
features = df_fe[[
    'Grupo_Edad',
    'Genero',
    'Categoria_Producto',
    'Metodo_Pago',
    'Shopping_Mall'
]]

In [ ]:
X_fe = df_fe.drop('Nivel_Gasto', axis=1)
y_fe = df_fe['Nivel_Gasto']

In [47]:
# División del dataset actualizado
X_fe_train, X_fe_test, y_fe_train, y_fe_test = train_test_split(
    X_fe, y_fe, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# ColumnTransformer
categorical_features = features.columns.tolist()

preprocessor_fe = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

### 12.4) Reconstruir Pipelines

In [50]:
# Logistic Regression

log_reg_pipeline_fe = Pipeline(steps=[
    ('preprocessor', preprocessor_fe),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [51]:
# Random Forest

rf_pipeline_fe = Pipeline(steps=[
    ('preprocessor', preprocessor_fe),
    ('classifier', RandomForestClassifier(random_state=42))
])


### 12.5) Entrenar y comparar nuevamente

In [52]:
log_reg_pipeline_fe.fit(X_fe_train, y_fe_train)
rf_pipeline_fe.fit(X_fe_train, y_fe_train)

y_pred_log_fe = log_reg_pipeline_fe.predict(X_fe_test)
y_pred_rf_fe = rf_pipeline_fe.predict(X_fe_test)

In [86]:
# Comparación de accuracy
pd.DataFrame({
    'Modelo': [
        'Logistic Regression (FE)',
        'Random Forest (FE)'
    ],
    'Accuracy': [
        accuracy_score(y_test, y_pred_log_fe),
        accuracy_score(y_test, y_pred_rf_fe)
    ]
})

,Modelo,Accuracy
0,Logistic Regression (FE),0.679771
1,Random Forest (FE),0.672129


La ingeniería de features permitió capturar patrones más relevantes del
comportamiento del cliente. Sin embargo, no hubo mejora en el accuracy 
para ambos modelos (para la regresión logística disminuyó).

Asimismo, una mejor clasificación del nivel de gasto permite segmentar clientes
de manera más efectiva y diseñar campañas comerciales más precisas.

## 13) Rebalanceo de clases y evaluación con F1-score

El siguiente paso lógico sería rebalancear las clases para mejorar el desempeño en todas las clases. Para ello primero debemos verificar el desbalanceo.

### Analizar distribución de clases

In [57]:
y_fe.value_counts(normalize=True)

Nivel_Gasto
Bajo     0.338287
Medio    0.331953
Alto     0.329761
Name: proportion, dtype: float64

- Las tres clases tienen proporciones muy similares (~33%)

- No hay desbalance de clases

- No existe una clase dominante

Como no hay desbalance, el siguiente paso con mayor impacto es:

✅ Random Forest optimizado + features mejoradas

## 14) Random Forest optimizado con features mejoradas

Se optimiza un modelo Random Forest utilizando features mejoradas
(grupos etarios y reducción de cardinalidad), con el objetivo de
incrementar la capacidad predictiva sin incurrir en overfitting.

Se utlizará RandomizedSearchCV porque:

- Es más eficiente

- El dataset es mediano

- Ya se conocen rangos razonables

In [58]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

### Para la configuración de hyperparámetros:

Priorizo max_depth y max_features porque controlan la complejidad y la
diversidad del bosque, mientras que n_estimators solo mejora estabilidad.
También priorizo min_samples_leaf porque tiene un impacto medio–alto, reduce ruido y suaviza decisiones.

In [66]:
param_dist = {
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__min_samples_leaf': randint(1, 8)
}


In [ ]:
# Configurar RandomizedSearchCV con n_iter = 15 porque tenemos 3 hyperpar. (n_iter ≈ 5 × número de hiperparámetros) 
# debe ser suficiente para cubrir el espacio de búsqueda sin evaluar combinaciones redundantes.
rf_random_search = RandomizedSearchCV(
    estimator=rf_pipeline_fe,   # pipeline con features mejoradas
    param_distributions=param_dist,
    n_iter=15,
    cv=5, # estándar mínimo recomendado, balance perfecto entre: bias, varianza y costo computacional
    scoring='accuracy',
    random_state=42,
    n_jobs=-1,
    verbose=2
)

In [68]:
rf_random_search.fit(X_fe_train, y_fe_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=5; total time=  24.0s
[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=5; total time=  24.3s
[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=5; total time=  24.0s
[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=5; total time=  23.9s
[CV] END classifier__max_depth=20, classifier__max_features=log2, classifier__min_samples_leaf=5; total time=  23.9s
[CV] END classifier__max_depth=20, classifier__max_features=sqrt, classifier__min_samples_leaf=5; total time=  24.0s
[CV] END classifier__max_depth=20, classifier__max_features=sqrt, classifier__min_samples_leaf=5; total time=  24.5s


/usr/local/python/3.12.1/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END classifier__max_depth=20, classifier__max_features=sqrt, classifier__min_samples_leaf=5; total time=  24.9s
[CV] END classifier__max_depth=20, classifier__max_features=sqrt, classifier__min_samples_leaf=5; total time=  24.1s
[CV] END classifier__max_depth=20, classifier__max_features=sqrt, classifier__min_samples_leaf=5; total time=  24.6s
[CV] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=2; total time=  26.0s
[CV] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=2; total time=  26.1s
[CV] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=2; total time=  26.0s
[CV] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=2; total time=  25.9s
[CV] END classifier__max_depth=None, classifier__max_features=sqrt, classifier__min_samples_leaf=2; total time=  26.0s
[CV] END classifier__max_depth=20, classifier__max_fea

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'classifier__max_depth': [None, 10, ...], 'classifier__max_features': ['sqrt', 'log2'], 'classifier__min_samples_leaf': <scipy.stats....x72a7d6fb10a0>}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",15
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies 

In [ ]:
# Mejores hiperparámetros
rf_random_search.best_params_

{'classifier__max_depth': 10,
 'classifier__max_features': 'sqrt',
 'classifier__min_samples_leaf': 2}

In [ ]:
# Evaluación del modelo optimizado
best_rf_fe = rf_random_search.best_estimator_

y_pred_rf_fe_opt = best_rf_fe.predict(X_fe_test)

accuracy_rf_fe_opt = accuracy_score(y_test, y_pred_rf_fe_opt)
accuracy_rf_fe_opt

0.680072390910919

In [89]:
# Comparación de modelos
pd.DataFrame({
    'Modelo': [
        'Random Forest (FE)',
        'Random Forest (FE + Optimizado)'
    ],
    'Accuracy': [
        accuracy_score(y_test, y_pred_rf_fe),
        accuracy_rf_fe_opt
    ]
})

,Modelo,Accuracy
0,Random Forest (FE),0.672129
1,Random Forest (FE + Optimizado),0.680072


La optimización de hiperparámetros permitió mejorar el desempeño del
Random Forest, aprovechando la capacidad del modelo para capturar
relaciones no lineales presentes en las features mejoradas.

## 15) Comparación final de modelos y conclusiones

In [90]:
# Comparación final
pd.DataFrame({
    'Modelo': [
        'Logistic Regression',
        'Logistic Regression (FE)',
        'Random Forest (FE Optimizado)'
    ],
    'Accuracy': [
        accuracy_score(y_test, y_pred_log),
        accuracy_score(y_fe_test, y_pred_log_fe),
        accuracy_rf_fe_opt
    ]
})

,Modelo,Accuracy
0,Logistic Regression,0.722652
1,Logistic Regression (FE),0.679771
2,Random Forest (FE Optimizado),0.680072


### Conclusiones:

- 1️⃣ Sobre el problema y el dataset

El objetivo fue clasificar el nivel de gasto de los clientes a partir de sus características demográficas y de compra utilizando el Customer Shopping Dataset.
El dataset presenta clases balanceadas, lo que permite usar accuracy como métrica principal sin sesgos significativos.

- 2️⃣ Evaluación de modelos

El mejor desempeño fue obtenido por Logistic Regression sin Feature Engineering, lo que indica que las variables originales contenían suficiente señal predictiva.

- 3️⃣ Impacto del Feature Engineering

Se aplicaron técnicas de Feature Engineering como:

    Agrupación de edad en grupos etarios

    Consolidación de categorías poco frecuentes en “Otros”

Resultado:

    No todo Feature Engineering mejora el modelo. A veces destruye información útil. En el caso del grupo etario,
perdí granularidad y el Logistic Regression sí aprovecha variables numéricas continuas. Por otro lado, respecto
a las categorías de productos, al agrupar categorías en “Otros” se obtuvo menos separabilidad y coeficientes menos claros.

    Por ende, se decidió mantener las variables originales para el modelo final.

- 4️⃣ Interpretación de los resultados

El accuracy alcanzado (~0.72) representa un límite razonable dado el tipo de variables disponibles.

Modelos más complejos como Random Forest no superaron a un modelo lineal bien configurado.

Esto sugiere que la complejidad del modelo no compensa la falta de información adicional en los datos.

- 5️⃣ Conclusión de negocio

El modelo permite:

    Segmentar clientes por nivel de gasto de forma razonablemente precisa

    Identificar patrones generales asociados al gasto

    Servir como base para estrategias de:

    Marketing segmentado

    Promociones dirigidas

    Priorización de clientes de mayor valor

Sin embargo, para mejorar significativamente la precisión sería necesario:

    Historial de compras por cliente

    Variables temporales (frecuencia, recurrencia)

    Información adicional del comportamiento del cliente

- El proyecto demuestra que la selección del modelo debe basarse en métricas y no en complejidad, y que redefinir o evaluar correctamente el problema es tan importante como entrenar modelos avanzados.

## 16) Exportación de modelo ganador

In [91]:
import pickle

In [ ]:
# Guardar el pipeline completo del LR
with open('../models/modelo_logistic_regression.pkl', 'wb') as file:
    pickle.dump(log_reg_pipeline, file)

In [97]:
# Cargar el modelo (validación)
with open('../models/modelo_logistic_regression.pkl', 'rb') as file:
    modelo_cargado = pickle.load(file)

In [98]:
# Probar predicción con el modelo cargado
pred_test = modelo_cargado.predict(X_test)
accuracy_score(y_test, pred_test)

0.7226523225417253